In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_white
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
main_reg = pd.read_csv('data_reg.csv')

In [ ]:
regr = 'stock_var'

In [ ]:
X = main_reg.loc[:,:'user_score']
X = sm.add_constant(X)
est = sm.OLS(main_reg.loc[:,regr], X).fit()

In [ ]:
est.summary(alpha = 0.1)

In [ ]:
# мультиколинеарность
pd.Series([variance_inflation_factor(X.values, i) for i in range(X.shape[1])], index = X.columns)

In [ ]:
#мутльтиколлинеарность
target = 'station'
var_list = main_reg.loc[:,:'user_score'].columns.tolist()
var_list.pop(var_list.index(target))
for i in var_list:
    print(i + ' ' + str(stats.pearsonr(main_reg.loc[:,target], main_reg.loc[:,i])[0]))

In [ ]:
# экзогенность
err = main_reg.loc[:,regr] - est.predict()
for i in main_reg.loc[:,:'user_score'].columns.tolist():
    print(i + ' ' + str(stats.pearsonr(main_reg.loc[:,i], err)[1]))

In [ ]:
# автокорелляция
temp = pd.DataFrame({regr: main_reg.loc[:,regr], 'err': err}).sort_values(regr)
fig = px.line(temp, x = regr, y = 'err')
fig.show()

In [ ]:
# автокорреляция
for j in range(1,51):
    print(str(j) + ' ' + str(stats.pearsonr([temp.err[i-j] for i in range(len(temp.err))][j:], temp.err[j:])[1]))

In [ ]:
# гомоскедастичность
fig = go.Figure(data = go.Scatter(x = main_reg.loc[:,regr], y = err, mode = 'markers'))
fig.show()

In [ ]:
het_white(err, main_reg.loc[:,:'station'])